## Preambule

In [14]:
import numpy as np
import xarray as xr
from importlib import reload
import Plotfunctions
import pandas as pd
import plotly.express as px
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import os
import matplotlib as mpl
import plotly.io
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import date
reload(Plotfunctions)
import plotly.graph_objects as go
from Plotfunctions import triangleplot, tableplot, spaghettiplot, draw_line
path = Path.cwd() / "Data"

## Parameters

In [15]:
timestamp = 2050
srt = 2 # 1=model, 2=ccat

## Prereading stuff

In [16]:
try:
    XRvar.close()
except:
    3
XRvar = xr.open_dataset(path / "Variances.nc")
XRvarsub = XRvar.sel(Time=timestamp)

In [17]:
Varlist = pd.read_excel(path / "Varlist_noadd.xlsx", sheet_name = "Data")
vars = np.array(Varlist["Variable"])
cats = np.array(Varlist["Category"])
Names = np.unique(cats)

In [18]:
XRraw = xr.open_dataset(path / "XRdata.nc")
XRmeta = xr.open_dataset(path / "XRmeta.nc")
DF = pd.read_csv(path / "Models.csv")
values = np.array(XRraw.Value)
modscens = np.array(XRraw.ModelScenario)
mods = np.array([i.split('|')[0] for i in modscens])
ccat = np.array(XRmeta.sel(ModelScenario=XRraw.ModelScenario).Category.data)
unimods = np.unique(mods)
uniccat = np.unique(ccat)

In [19]:
XR = XRvar.sel(Variable = vars[cats == "Primary Energy"])
DF_counts = pd.read_csv("X:/user/dekkerm/Projects/AR6_Variance/variancedecomposition/Data/Counts.csv", index_col=0)
XR = XR.sel(Time=range(2030,2101))
varlist = XR.Variable.data
years = XR.Time.data
varmax = np.max(XR['Var_total']).data
cols = ['forestgreen', 'tomato', 'steelblue', 'goldenrod', 'purple', 'grey', 'brown',
        'magenta', 'red', 'darkgrey', 'blue', 'black', 'darkgreen']
years_str = np.copy(years).astype(str)
years_str[(years_str != '2050') & (years_str != '2100')] = ''

In [20]:
tot = np.array(XR.sel(Time=2050).S_c)+np.array(XR.sel(Time=2050).S_z)+np.array(XR.sel(Time=2050).S_m)
ar = np.array([np.array(XR.Variable),
np.array(XR.sel(Time=timestamp).S_m/tot).round(2),
np.array(XR.sel(Time=timestamp).S_c/tot).round(2),
np.array(XR.sel(Time=timestamp).S_z/tot).round(2)])

colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
bar = px.colors.diverging.RdBu_r
colors = []
for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
ar100 = (ar[1:]*100).astype(int)

In [21]:
def rescale_size(s, max):
    s /= max
    s *= 1e11
    s = s ** 0.1
    return s

## Create traces

In [22]:
varmax = np.max(XRvarsub['Var_total']/XRraw.sel(Time=timestamp).Value.mean(axis=0)).data
scatters = []
tables = []
namecolors = ['goldenrod', 'violet', 'forestgreen', 'silver', 'teal', 'peru']
namesymbols = ['circle', 'diamond-wide', 'star', 'square', 'cross', 'star-triangle-up', 'diamond-tall']
for n_i, name in enumerate(['Emissions', 'Primary Energy', 'Electricity Generation', 'Sector Industry', 'Sector Transportation', 'Sector Residential']):
    # General stuff
    xrsub = XRvarsub.sel(Variable = vars[cats == name])

    # Table
    tot = np.array(xrsub.S_c)+np.array(xrsub.S_z)+np.array(xrsub.S_m)
    var = np.array(xrsub.Variable)
    if name == 'Electricity Generation':
        for v_i in range(len(var)):
            if var[v_i] == 'Secondary Energy|Electricity':
                var[v_i] = 'Total'
            else:
                var[v_i] = var[v_i][29:]
    if name == 'Primary Energy':
        for v_i in range(len(var)):
            var[v_i] = var[v_i][15:]
    if name == 'Emissions':
        for v_i in range(len(var)):
            var[v_i] = var[v_i][10:]
    if name == 'Sector Industry':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Industry':
                var[v_i] = 'Total'
            else:
                var[v_i] = var[v_i][22:]
    if name == 'Sector Residential':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Residential and Commercial':
                var[v_i] = 'Total'
            else:
                var[v_i] = var[v_i][40:]
    if name == 'Sector Transportation':
        for v_i in range(len(var)):
            if var[v_i] == 'Final Energy|Transportation':
                var[v_i] = 'Total'
            else:
                var[v_i] = var[v_i][28:]
    ar = np.array([var,
                   np.array(xrsub.S_m/tot).round(2),
                   np.array(xrsub.S_c/tot).round(2),
                   np.array(xrsub.S_z/tot).round(2)])
    colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 100, colortype='rgb')
    bar = px.colors.diverging.RdBu_r
    colors = []
    for i in range(len(bar)-1):
        colors = colors+n_colors(px.colors.diverging.RdBu_r[i], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
    colors = colors+n_colors(px.colors.diverging.RdBu_r[i+1], px.colors.diverging.RdBu_r[i+1], int(100/len(bar)), colortype='rgb')
    ar100 = 10+(ar[1:]*70).astype(int)

    tables.append(go.Table(columnwidth = [4, 1, 1, 1],
                        header = dict(values = [['<b>'+name+'</b>'],
                                      ["<b>Model</b>"],
                                      ['<b>C-cat</b>'],
                                      ['<b>Other</b>']],
                                      line_color='white',
                                      fill_color=[namecolors[n_i], 'white', 'white', 'white'],
                                      align=['center','center','center','center'],
                                      font=dict(color='black'),
                                     ),
                        cells=dict(values=ar[:, ar[srt].argsort()[::-1]],
                                   line={'color': 'black', 'width': 0},
                                   align=['center','center','center','center'],
                                   fill_color=['white', 
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][0]],
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][1]],
                                               np.array(colors)[ar100[:, ar[srt].argsort()[::-1]][2]]],
                                   font={#'family' : ["Arial", "Arial Black", "Arial Black", "Arial Black"],
                                         'color': 'black',
                                         'size': 9},
                                   height=18
                                  )
                       ))

    # Scatter
    varlist = np.array(xrsub.Variable)
    for v in range(len(varlist)):
            ds = xrsub.sel(Variable=varlist[v])
            a, b, c, s = np.array(ds[['S_c', 'S_m', 'S_z', 'Var_total']].to_array())
            s = 20# 2*rescale_size(float(s/XRraw.sel(Time=timestamp, Variable=varlist[v]).Value.mean(axis=0)), varmax)
            lw = 1 # s/10
            if varlist[v] in ['Primary Energy|Coal', 'Primary Energy|Oil', 'Emissions|Kyoto Gases', 'Secondary Energy|Electricity|Coal|w/o CCS',
                              'Final Energy|Industry|Electricity', 'Final Energy|Transportation|Electricity', 'Final Energy|Residential and Commercial|Electricity']:
                lw = 4
            scatters.append(go.Scatterternary(a=[a], b=[b], c=[c],
                                            mode='markers', name=varlist[v]+' ('+str(list(DF_counts[DF_counts.Variable==varlist[v]].Count)[0])+')', showlegend=False, text=[years[-1]],
                                            hovertemplate='%{text}<br>Var (climate): %{a}<br>Var (model): %{b} <br>Var (other): %{c}',
                                            marker={'size': s, 'symbol': namesymbols[n_i], 'color': namecolors[n_i], 'opacity': 1, 'line' :dict(width=lw, color='black')},
                                            line={'width': 1.5},
                                            textfont=dict(size=1, color=namecolors[n_i])))
                                
        

Ternary plot

In [23]:
trace_ternary_lines = []
for i in range(10):
        a = [0+i*0.1, 0+i*0.1]
        b = [0, 1-a[0]]
        c = [1-a[0], 0]
        rgb = mpl.colors.colorConverter.to_rgb(plt.cm.get_cmap('Greys')(0.+i/20))
        col = 'rgb('+str(rgb[0])+','+str(rgb[1])+','+str(rgb[2])+')'
        trace_ternary_lines.append(go.Scatterternary(a=a, b=b, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        trace_ternary_lines.append(go.Scatterternary(a=b, b=a, c=c, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))
        trace_ternary_lines.append(go.Scatterternary(a=b, b=c, c=a, showlegend=False, mode='lines', hoverinfo='skip', line={'width': 0.25+i*0.1, 'color': col}))

trace_ternary_border = []
trace_ternary_border.append(go.Scatterternary(a=[0.005, 0.005], b=[0, 1], c=[1, 0], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))
trace_ternary_border.append(go.Scatterternary(a=[0, 1], b=[0.005, 0.005], c=[1, 0], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))
trace_ternary_border.append(go.Scatterternary(a=[0, 1], b=[1, 0], c=[0.005, 0.005], showlegend=False, mode='lines', hoverinfo='skip', line={'width': 2, 'color': 'black'}))

## Add together

In [24]:
def draw_line(fig, name, x, y, xt, yt, color):
    # fig.add_shape(type="line", xref='paper', yref='paper',
    #                 x0=x,
    #                 y0=x+xt,
    #                 x1=x,
    #                 y1=y+yt,
    #                 line=dict(color=color, width=3))
    fig.add_annotation(xref='paper',
                       yref='paper',
                       #xref = 'x4',
                       #yref = 'y4',
                       x=x,
                       ax=xt,
                       align="center",
                       y=y,
                       ay=yt,
                       text=name,
                       #textangle=90,
                       font=dict(color=color,
                                 family='Arial Black',
                                 size=12),
                       showarrow=True)

In [25]:
fig = make_subplots(
    rows=6, cols=4,
    horizontal_spacing = 0.02,
    vertical_spacing=0,
    # column_widths = [0.15, 0.7, 0.15],
    # row_heights = [0.15, 0.7, 0.15],
    specs = [[{"type": "table", "rowspan": 2}, {"type": "scatterternary", "colspan": 2, "rowspan": 6}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}, {}],
             [{"type": "table", "rowspan": 2}, {}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}, {}],
             [{"type": "table", "rowspan": 2}, {}, {}, {"type": "table", "rowspan": 2}],
             [{}, {}, {}, {}]
            ]
)

fig.add_trace(tables[0], 1, 1)
fig.add_trace(tables[1], 3, 1)
fig.add_trace(tables[5], 5, 1)

fig.add_trace(tables[3], 1, 4)
fig.add_trace(tables[4], 3, 4)
fig.add_trace(tables[2], 5, 4)

for n in range(len(trace_ternary_lines)):
    fig.add_trace(trace_ternary_lines[n], 1, 2)
for n in range(len(trace_ternary_border)):
    fig.add_trace(trace_ternary_border[n], 1, 2)
for n in range(len(scatters)):
    fig.add_trace(scatters[n], 1, 2)
fig.update_layout(height=870, width=1620)
fig.update_layout({#'margin':dict(l=0,r=0,b=0,t=0),
'ternary':
{'sum':1,
'bgcolor':'whitesmoke',
'aaxis':{'title': 'Climate target<br>', 'min': 0, 
        'linewidth':0, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'], 'tickfont':{'size':12}},
'baxis':{'title': 'Model &nbsp; &nbsp;', 'min': 0, 
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':60, 'tickfont':{'size':12}},
'caxis':{'title': 'Other scenario<br>assumptions', 'min': 0, 
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], 'ticktext':['50%', '60%', '70%', '80%', '90%', '100%'],'tickangle':-60, 'tickfont':{'size':12}}}})

draw_line(fig, "Primary energy coal", 0.5, 0.725, 150, -40, "violet")
draw_line(fig, "Primary energy oil", 0.49, 0.62, 210, -40, "violet")
draw_line(fig, "Kyoto gas emissions", 0.51, 0.905, 120, -30, "goldenrod")
draw_line(fig, "Coal (w/o CCS) in<br>electricity production", 0.52, 0.765, 150, -60, "forestgreen")
draw_line(fig, "Electricity usage<br>in industry", 0.385, 0.235, -120, -120, "silver")
draw_line(fig, "Electricity usage<br>in transport", 0.395, 0.205, 200, -40, "teal")
draw_line(fig, "Electricity usage<br>in residential", 0.38, 0.145, 180, 40, "peru")
fig.show()

## Savings

In [26]:
try:
    os.remove('Figures/Paperfigures/Figure_3.html')
except:
    3
def html_w(typ):
    return '<html> '+typ+' <p style="font-family: Arial">'

fig.write_image("Figures/Paperfigures/Figure_3.pdf")
fig.write_image("Figures/Paperfigures/Figure_3.png", scale=3)
today = date.today()
td = today.strftime("%B %d, %Y")

with open('Figures/Paperfigures/Figure_3.html', 'a') as f:
    f.write(html_w('<h1>')+'Figure 3 - Overview</p></h1>')
    f.write(html_w('<body>')+'Data is for year '+str(timestamp)+'. Version: '+td+'</p></body>')
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))